In [56]:
!pip install clickhouse-connect
!pip install clickhouse-driver


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [57]:
!pwd

/app/workspace/options_transform


In [117]:
import os,h5py
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from IPython.display import clear_output
import joblib
import random
import hashlib
import uuid
from tqdm.notebook import  tqdm

In [62]:
class Scaler:
    def __init__(self, keys, create=False, path="./data/scaler/"):
        self.scaler = {}
        # self._scale = scale
        self._keys = keys
        self._path = path
        self._creatScaler(create)

    def __call__(self):
        return self.scaler

    @property
    def keys(self):
        return self._keys

    def _creatScaler(self, create):
        if not os.path.exists(self._path):
            os.makedirs(self._path)

        dir_list = os.listdir(self._path)

        for k in self._keys:
            self.scaler[k] = None
            for dir in dir_list:
                if k in dir:
                    print(f"[Load-{k}] : ", self._path + dir)
                    self.scaler[k] = joblib.load(self._path + dir)

            if k not in self.scaler.keys() or create:
                print(
                    f"[Set] : [{k}] - {StandardScaler()} ",
                )
                self.scaler[k] = StandardScaler()

    def save(self):
        for k in self.scaler.keys():
            joblib.dump(self.scaler[k], self._path + f"scaler_{k}.gz")

    def groupTransform(self, c, data, **kwargs):
        # ------------------ customize --------------------
        if c in kwargs["QUOTE_COL"]:
            return self.scaler["QUOTE"].transform(data)
        elif c in kwargs["VEGA_COL"]:
            return self.scaler["VEGA"].transform(data)
        elif c in kwargs["VOLUME_COL"]:
            return self.scaler["VOLUME"].transform(data)
        else:
            return self.scaler[c].transform(data)

In [78]:
CSV_PATH     = '/app/raw/OptionsEOD.csv/'
##create @ part 1
##PARQUET_PATH = './data/OptionsEOD.parquet'
##create @ part 2
##PARQUET_STG_PATH = './data/OptionsEOD_STG.parquet'
SCALER_COL  = [ 'UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE','INTRINSIC_VALUE','DTE','C_VEGA','P_VEGA',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
#create @ part 3
OTHER_COL = ['SYMBOL','QUOTE_UNIXTIME', 'EXPIRE_UNIX']
DATETIME_COLUMNS = ['EXPIRE_DATE', 'QUOTE_DATE', 'QUOTE_READTIME']
export_dtype = {
    'SYMBOL':str,
    'QUOTE_UNIXTIME':int,
    'EXPIRE_UNIX':int,
    'UNDERLYING_LAST':float,
    'STRIKE':float,
    'STRIKE_DISTANCE':float,
    'INTRINSIC_VALUE':float,
    'DTE':float,
    'C_VEGA':float,
    'P_VEGA':float,	
    'C_BID':float,	
    'C_ASK':float, 
    'C_VOLUME':float,  
    'P_BID':float,	
    'P_ASK':float, 
    'P_VOLUME':float
}

QUOTE_COL = ['C_BID',	'C_ASK',  'P_BID',	'P_ASK']
VEGA_COL =  ["C_VEGA","P_VEGA"] 
VOLUME_COL =  ["TOTAL_VOLUME","C_VOLUME","P_VOLUME"] 

SCALER_QUOTE_COL = [i for i,v in enumerate(SCALER_COL) if v in QUOTE_COL]
SCALER_VEGA_COL = [i for i,v in enumerate(SCALER_COL) if v in VEGA_COL]
SCALER_VOLUME_COL = [i for i,v in enumerate(SCALER_COL) if v in VOLUME_COL]
SCALER_OTHER_COL = [i for i,v in enumerate(SCALER_COL) if v not in QUOTE_COL+VEGA_COL+VOLUME_COL ]


UNIQUE_KEYS = ['SYMBOL','EXPIRE_DATE']
H5_PATH = './data/OptTrainRenforcement/'
START = True#True#'2010-09'#True
BUCKET = 'tensorflow'

MAX_OPTIONS_LEN = 16
RM_STRIKE_ = 2
#==== Create options_qoute
options_qoute = {}

In [20]:
#example data
EOD_CSV = pd.read_csv(CSV_PATH+"qqq/qqq_eod_201201.txt", engine='pyarrow')
EOD_CSV.head()

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],...,[P_LAST],[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT]
0,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.003130,0.002320,0.000350,-0.004080,0.000000,0.793590,,10.9,0.192
1,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.003580,0.002780,0.000530,-0.004420,-0.000420,0.721270,,9.9,0.174
2,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.004040,0.003340,0.000950,-0.004350,-0.000430,0.648640,,8.9,0.156
3,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.010000,-0.004390,0.004150,0.001170,-0.004050,-0.000330,0.577770,0.000000,7.9,0.139
4,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,0.971540,0.016360,...,0.010000,-0.004860,0.005270,0.001370,-0.004140,0.000000,0.507560,0.000000,6.9,0.121


In [21]:
#create 
import clickhouse_connect  
clickhouse_client = clickhouse_connect.get_client(host='host.containers.internal', port=8123, database='stagging')

In [ ]:
############## STAGGING #################

In [24]:
def transform_I() 
    for d in os.listdir(CSV_PATH) :
        for f in tqdm( os.listdir(CSV_PATH+f"{d}/" )[:] , desc=f"SYMBOL-{d}" ):
            if f.endswith(".txt"):
                ## load
                print( f"[LOAD] : {CSV_PATH}{d}/{f}        ",end='\r')
                
                EOD_CSV = pd.read_csv(CSV_PATH+f"{d}/"+f, engine='pyarrow' )
                    
                ## rename col.
                for c in EOD_CSV.columns:
                    EOD_CSV = EOD_CSV.rename( columns={ c:c.strip().replace(']','').replace('[','') } )
                
                ## add symbol 
                EOD_CSV['SYMBOL'] = d.upper()
                
                ## add INTRINSIC_VALUE
                EOD_CSV['INTRINSIC_VALUE'] = EOD_CSV['UNDERLYING_LAST'] - EOD_CSV['STRIKE']
                
                EOD_CSV.dropna(subset=SCALER_COL)
    
    
                # date columns convert to datetime
                for c in ["QUOTE_READTIME","QUOTE_DATE","EXPIRE_DATE"]:
                    EOD_CSV[c] = pd.to_datetime(EOD_CSV[c])
                # # Convert datetime columns to string using strftime
                # EOD_CSV['EXPIRE_DATE'] = EOD_CSV['EXPIRE_DATE'].dt.strftime('%Y-%m-%d')
                # EOD_CSV['QUOTE_DATE'] = EOD_CSV['QUOTE_DATE'].dt.strftime('%Y-%m-%d')
                # EOD_CSV['QUOTE_READTIME'] = EOD_CSV['QUOTE_READTIME'].dt.strftime('%Y-%m-%d %H:%M:%S')
                
    
                
                #clean float data
                for c in ['INTRINSIC_VALUE','C_DELTA','C_GAMMA','C_VEGA','C_THETA','C_RHO','C_IV','C_VOLUME','C_LAST','C_BID','C_ASK','STRIKE','P_BID','P_ASK','P_LAST','P_DELTA','P_GAMMA','P_VEGA','P_THETA','P_RHO','P_IV','P_VOLUME','STRIKE_DISTANCE','STRIKE_DISTANCE_PCT']:
                    if EOD_CSV[c].dtype not in ( 'float32','float64'):
                        EOD_CSV[c] = EOD_CSV[c].apply(lambda x: x.strip())
                        EOD_CSV[c] = EOD_CSV[c].replace('', np.nan).fillna(np.nan)
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                    if EOD_CSV[c].dtype == 'float32':
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
    
                # Convert any necessary columns to the appropriate types for ClickHouse
                EOD_CSV['C_SIZE'] = EOD_CSV['C_SIZE'].astype(str)
                EOD_CSV['P_SIZE'] = EOD_CSV['P_SIZE'].astype(str)
                EOD_CSV['SYMBOL'] = EOD_CSV['SYMBOL'].astype(str)
                
                
                # For columns that need UInt32 or UInt64, also handle NaN or invalid values
                EOD_CSV['QUOTE_UNIXTIME'] = EOD_CSV['QUOTE_UNIXTIME'].fillna(0).astype('UInt32')  # Fill NaN with 0 for UInt32 columns
                EOD_CSV['EXPIRE_UNIX'] = EOD_CSV['EXPIRE_UNIX'].fillna(0).astype('UInt32')
    
                EOD_CSV['C_VOLUME'] = EOD_CSV['C_VOLUME'].fillna(0).astype('UInt64')  # Convert to UInt64
                EOD_CSV['P_VOLUME'] = EOD_CSV['P_VOLUME'].fillna(0).astype('UInt64')
                
                # For columns that should be of type float64 (for example, 'C_LAST', 'DTE'), you can use:
                EOD_CSV['DTE'] = EOD_CSV['DTE'].fillna(0).astype('float64')
                EOD_CSV['C_LAST'] = EOD_CSV['C_LAST'].fillna(0).astype('float64')
    
                # Convert the DataFrame to CSV format (without the index)
                #csv_data = EOD_CSV.to_csv(index=False)
                # Insert data into ClickHouse using the CSV format
                clickhouse_client.insert('stagging.options_trading', EOD_CSV)


SYMBOL-qqq:   0%|          | 0/145 [00:00<?, ?it/s]

SYMBOL-spy:   0%|          | 0/169 [00:00<?, ?it/s]

SYMBOL-spx:   0%|          | 0/168 [00:00<?, ?it/s]

In [ ]:
############## DATAWAREHOUSE #################

In [124]:
SCALER = Scaler(
    [v for v in SCALER_COL if v not in QUOTE_COL+VEGA_COL+VOLUME_COL ] + ['QUOTE','VEGA','VOLUME']
    , create=True
    , path="./data/scaler/")

[Set] : [UNDERLYING_LAST] - StandardScaler() 
[Set] : [STRIKE] - StandardScaler() 
[Set] : [STRIKE_DISTANCE] - StandardScaler() 
[Set] : [INTRINSIC_VALUE] - StandardScaler() 
[Set] : [DTE] - StandardScaler() 
[Set] : [QUOTE] - StandardScaler() 
[Set] : [VEGA] - StandardScaler() 
[Set] : [VOLUME] - StandardScaler() 


In [125]:
# Query ClickHouse and load data into a Pandas DataFrame
query = """SELECT QUOTE_DATE,SYMBOL,EXPIRE_DATE FROM stagging.options_trading
GROUP BY QUOTE_DATE,SYMBOL,EXPIRE_DATE
ORDER BY SYMBOL,EXPIRE_DATE,QUOTE_DATE"""  # Replace 'your_table' with your table name
data = clickhouse_client.query(query).result_rows  # Fetch data as a list of tuples
column_names = clickhouse_client.query(query).column_names  # Fetch column names

# Create a DataFrame from the result
DF_UNIQUE_KEYS = pd.DataFrame(data, columns=column_names)

MAX_OPTION_LEN = 16
STRIKE_SELECT = []
ID = None
ref_key_symbol = None
ref_key_expire = None

In [ ]:
for i,keys in tqdm( DF_UNIQUE_KEYS[:].iterrows() ,total = len(DF_UNIQUE_KEYS) ):
    key_quote = keys['QUOTE_DATE'].strftime('%Y-%m-%d')
    key_symbol = keys['SYMBOL']
    key_expire = keys['EXPIRE_DATE'].strftime('%Y-%m-%d')
    
    query = f"""SELECT * FROM stagging.options_trading
            WHERE
            QUOTE_DATE = '{key_quote}'
            AND SYMBOL = '{key_symbol}'
            AND EXPIRE_DATE = '{key_expire}'
            ORDER BY STRIKE  desc """ 
    data = clickhouse_client.query(query).result_rows  # Fetch data as a list of tuples
    column_names = clickhouse_client.query(query).column_names  # Fetch column names
    DF = pd.DataFrame(data, columns=column_names)

    #select strike
    if not len(STRIKE_SELECT)  or (key_symbol,key_expire) == (ref_key_symbol,ref_key_expire):
        STRIKE_SELECT = DF[ DF['INTRINSIC_VALUE'].abs().isin(DF['INTRINSIC_VALUE'].abs().sort_values()[:MAX_OPTION_LEN]) ]['STRIKE'].values
        ID = uuid.uuid4()
        SCALER.save()
        clear_output(wait = False)

    rm_strike_index = DF[ ~DF['STRIKE'].isin(STRIKE_SELECT) ].index
    DF = DF.drop(rm_strike_index)

    #break loops when df < X
    if len(DF) < 5 : 
        print(f"""{key_symbol}|{key_expire}|{key_quote} -> break (less rows)""")
        ref_key_symbol = key_symbol
        ref_key_expire = key_expire
        continue 
        
    if len(DF) == MAX_OPTION_LEN and not np.isnan( np.sum( DF[SCALER_COL].values ) ) :
        DATA  = np.empty((0,) + ( MAX_OPTION_LEN,len(SCALER_COL) ) )
        DATA = np.vstack((DATA ,[DF[SCALER_COL]]))
        
        if len(DATA) :
            #========================== partial_fit ================================#
            for col_i in SCALER_OTHER_COL_INDEX:
                SCALER()[SCALER_COL[col_i]].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,[col_i]] )
            SCALER()['QUOTE'].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,SCALER_QUOTE_COL].reshape(-1, 1) )
            SCALER()['VEGA'].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,SCALER_VEGA_COL].reshape(-1, 1) )
            SCALER()['VOLUME'].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,SCALER_VOLUME_COL].reshape(-1, 1) )
            
    DF['ID'] = ID
    clickhouse_client.insert('warehouse.options_trading', DF)
    ref_key_symbol = key_symbol
    ref_key_expire = key_expire
    
SCALER.save()

In [135]:
query = f"""SELECT * FROM stagging.options_trading
        WHERE
        QUOTE_DATE = '{key_quote}'
        AND SYMBOL = '{key_symbol}'
        AND EXPIRE_DATE = '{key_expire}'
        ORDER BY STRIKE  desc """ 
data = clickhouse_client.query(query).result_rows  # Fetch data as a list of tuples
column_names = clickhouse_client.query(query).column_names  # Fetch column names
DF = pd.DataFrame(data, columns=column_names)